In [216]:
import sys
import random
import PySide6
from PySide6 import QtCore, QtWidgets, QtGui
from PySide6.QtWidgets import *
from PySide6.QtCore import Qt
from PySide6.QtGui import QPalette, QColor
import numpy as np
import pandas as pd


In [217]:
data=None

try:
    data=pd.read_feather("data/stage2_partial.feather")
except:
    try:
        print("Loading backup, main data corrupted.")
        data=pd.read_feather("data/stage2_partial.backup.feather")
    except:
        print("Loading stage 1, main data & backup corrupted.")
        data=pd.read_feather("data/stage1.feather")


In [218]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20169 entries, 0 to 20168
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   20168 non-null  object
 1   labels  20168 non-null  object
 2   time    20168 non-null  object
dtypes: object(3)
memory usage: 472.8+ KB


In [219]:
def save(is_backup=False):
    data.to_feather(f"data/stage2_partial{'.backup' if is_backup else ''}.feather")
save(True)

In [220]:
data[data["labels"]==""].index

Index([   47,    76,    96,   129,   210,   324,   501,   631,   732,   787,
       ...
       19571, 19577, 19579, 19733, 19828, 20020, 20045, 20046, 20110, 20166],
      dtype='int64', length=491)

In [221]:
def get_next():
    idx=data[data["labels"]==""].index
    if(idx.shape[0]==0):
        return None, 0, None
    return idx[0],idx.shape[0], data.loc[idx[0]]

In [222]:
next_idx,remainder,row=get_next()

In [223]:
def safe_update(idx, value, time):
    save(True)
    data.loc[idx,"labels"]=value
    data.loc[idx,"time"]=time
    save(False) 

In [224]:
keys = ["NO_VOLCAN", "INACTIVO", "FUMAROLA", "ERUPCION"]
labels = {
    keys[0]:"NO VOLCÁN",
    keys[1]:"INACTIVO",
    keys[2]:"FUMAROLA",
    keys[3]:"ERUPCIÓN",
}
labels_n = { key:val+" [N]" for key,val in labels.items()}

In [225]:
window=None

In [226]:
class ImageDisplay(QWidget):

    def loadpixmap(self, path):
        self.pixmap=QtGui.QPixmap(path)#.scaledToHeight(self.geometry().width())
        self.label.setPixmap(self.pixmap)
        
    def __init__(self, path = ""):
        super().__init__()
        label=QLabel()
        label.setAlignment(Qt.Alignment.AlignCenter)
        label.setScaledContents(True)
        self.label=label
        self.loadpixmap(path)
        
        self.layout=QGridLayout()
        self.layout.addWidget(label, 0,0)
        self.setLayout(self.layout)

In [227]:
class DataLayout(QWidget):

    def button_pressed(self, key, time):
        global next_idx,remainder,row
        safe_update(next_idx,key, time)
        next_idx,remainder,row=get_next()
        window.image.loadpixmap(row["image"])
        self.progress.setText(f"Remaining: {remainder} ({100*(remainder/data.shape[0]):.02F}%)")

    def go_back(self):
        global next_idx,remainder,row
        prev_idx=data[data["labels"]!=""].index[-1]
        safe_update(prev_idx,"","")
        next_idx,remainder,row=get_next()
        window.image.loadpixmap(row["image"])
        self.progress.setText(f"Remaining: {remainder} ({100*(remainder/data.shape[0]):.02F}%)")

    def get_callback(self,key,time):
        return lambda: self.button_pressed(key, time)
    
    def __init__(self):
        super().__init__()
        #self.setMinimumWidth(600)
        #self.setMaximumWidth(600)
        self.setSizePolicy(QSizePolicy.Minimum, QSizePolicy.MinimumExpanding)
        layout=QGridLayout()

        self.progress = QLabel(f"Remaining: {remainder} ({100*(remainder/data.shape[0]):.02F}%)")
        #self.progress.setMaximumHeight(50)
        self.progress.setSizePolicy(QSizePolicy.MinimumExpanding, QSizePolicy.Maximum)
        self.progress.setAlignment(Qt.Alignment.AlignCenter)
        layout.addWidget(self.progress, 0, 0, 1, 2)

        backtrack=QPushButton("ANTERIOR")
        backtrack.setSizePolicy(QSizePolicy.MinimumExpanding, QSizePolicy.Maximum)
        backtrack.setMinimumHeight(75)
        backtrack.clicked.connect(self.go_back)
        layout.addWidget(backtrack, 1, 1, 1, 1)
        
        for i,key in enumerate(keys):
            button=QPushButton(labels[key])
            button.setSizePolicy(QSizePolicy.MinimumExpanding, QSizePolicy.Maximum)
            button.setMinimumHeight(75)
            #button.setMinimumHeight(50)
            # set specific key, force it to stay fixed each loop
            button.clicked.connect(self.get_callback(key, "day"))
            if i==0:
                layout.addWidget(button, i*2+1, 0, 1, 1)
            else:
                button2=QPushButton(labels_n[key])
                button2.setMinimumHeight(75)
                button2.setSizePolicy(QSizePolicy.MinimumExpanding, QSizePolicy.Maximum)
                #button2.setMinimumHeight(50)
                
                button2.clicked.connect(self.get_callback(key, "night"))
                layout.addWidget(button, i*2+1, 0, 1, 1)
                layout.addWidget(button2,i*2+1, 1, 1, 1)
            
        self.setLayout(layout)

In [228]:
class MainWindow(QMainWindow):

    def __init__(self):
        super(MainWindow, self).__init__()

        self.setWindowTitle("Classifier")

        layout = QHBoxLayout()
        left = ImageDisplay(row["image"])
        right= DataLayout()

        self.data=right
        self.image=left

        layout.addWidget(left)
        layout.addWidget(right)

        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)

In [ ]:
if __name__ == "__main__" and remainder>0:
    if not QtWidgets.QApplication.instance():
        app = QtWidgets.QApplication([])
    else:
        app = QtWidgets.QApplication.instance()
    font=QtGui.QFont("Segoe UI", 14)
    app.setFont(font)
    app.setStyle(QStyleFactory.create("windows"))
    window = MainWindow()
    window.show()#Maximized()

    app.exec()

In [213]:
data[data["labels"]!=""]["labels"].value_counts()

labels
FUMAROLA     8382
ERUPCION     6809
NO_VOLCAN    3831
INACTIVO     1146
Name: count, dtype: int64

In [214]:
save(True)

In [215]:
save(False)